In [53]:
import os
import sys
sys.path.append('/Users/sonle/Documents/work/WB/')
import glob
import json
from tqdm.notebook import tqdm
import gc
import warnings
warnings.filterwarnings('ignore')


import pandas as pd
import numpy as np
import xarray as xr
import rioxarray as rxr 
import geopandas as gpd
import rasterio
from rasterio import Affine
from rasterstats import zonal_stats
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import linearmodels as lm
from scipy.stats import spearmanr


from wb_pimpam_ged.dhelper import get_daterange

In [186]:
gdp = xr.open_dataset('/Users/sonle/Documents/work/WB/GED/validation/GDP_per_capita_PPP_1990_2015_v2.nc', decode_coords=True)
gdp

KeyboardInterrupt: 

In [ ]:
# level = 1
# poly = gpd.read_file(f'/Users/sonle/Documents/work/WB/GED/SRB/shapefiles/gadm41_SRB_{level}.json')

# if level == 2:
#     poly = poly.drop(['GID_0', 'COUNTRY', 'NL_NAME_1', 'VARNAME_2', 'NL_NAME_2',
#                     'TYPE_2', 'ENGTYPE_2', 'CC_2', 'HASC_2'], axis=1)

# if level == 1:
#     poly = poly.drop(['GID_0', 'COUNTRY', 'NL_NAME_1', 'VARNAME_1',
#                     'TYPE_1', 'ENGTYPE_1', 'CC_1', 'HASC_1', 'ISO_1'], axis=1)
    


In [ ]:
poly = gpd.read_file(f'/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/shapefiles/SRB_03M_LVL3.geojson')
poly = poly.drop(['id', 'LEVL_CODE', 'CNTR_CODE', 'NUTS_NAME', 'MOUNT_TYPE', 'URBN_TYPE',
                  'COAST_TYPE', 'FID'], axis=1)
poly.head()

,NUTS_ID,NAME_LATN,geometry
0,RS222,Braničevska oblast,"POLYGON ((21.82202 44.08251, 21.80861 44.09600..."
1,RS223,Zaječarska oblast,"POLYGON ((22.33758 43.38527, 22.34016 43.41702..."
2,RS224,Jablanička oblast,"POLYGON ((22.22691 43.08012, 22.25253 43.06249..."
3,RS221,Borska oblast,"POLYGON ((22.53615 44.04551, 22.49600 44.04771..."
4,RS214,Moravička oblast,"POLYGON ((20.29580 43.31169, 20.28285 43.32850..."


In [ ]:
# df_list = []

# for y in [2014, 2015]:

#     df_poly = poly.copy(deep=True)

#     gdp.sel(time=y)['GDP_per_capita_PPP'].rio.to_raster('temp_file.tif', 
#                                                         engine='GeoTIFF')
#     df_poly[f'gdp_ppp'] = pd.DataFrame(zonal_stats(poly.geometry, 'temp_file.tif', stats=f'sum'))
#     df_poly['year'] = str(y)

#     df_list.append(df_poly)

# df_gdp_ppp = pd.concat(df_list, axis=0).reset_index(drop=True).drop(['geometry'], axis=1)
# df_gdp_ppp.head()



In [ ]:
path_list = glob.glob('/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/luminosity_alt_eu/*.tif')
path_list.sort()

In [ ]:
def_affine = rasterio.open(path_list[0]).profile['transform']
def_affine

Affine(0.0008983152841195215, 0.0, 18.858332759521115,
       0.0, -0.0008983152841195215, 46.188676963573435)

In [ ]:
def find_index(lst, substring):
    for i, s in enumerate(lst):
        if substring in s:
            return i
    return -1

print(find_index(path_list, '2014'))
print(find_index(path_list, '2019'))

print(path_list[find_index(path_list, '2014')].find('2014'))


0
5
80


In [ ]:
df_list = list()

for path in tqdm(path_list):
    
    df_poly = poly.copy(deep=True)

    df_poly[f'gdp_alt'] = pd.DataFrame(zonal_stats(poly.geometry, rasterio.open(path).read(1), affine=def_affine, stats='sum'))
    df_poly['year'] = path[80:84]

    df_list.append(df_poly)

    del df_poly

gc.collect()

df_gdp_alt = pd.concat(df_list, axis=0).reset_index(drop=True).drop(['geometry'], axis=1)
df_gdp_alt.columns = df_gdp_alt.columns.str.upper()
df_gdp_alt['YEAR'] = df_gdp_alt['YEAR'].astype(int)
df_gdp_alt = df_gdp_alt.rename({'GDP_ALT': 'GDP_ALT_MIL_USD_2017'}, axis=1)
df_gdp_alt.head()


  0%|          | 0/6 [00:00<?, ?it/s]

,NUTS_ID,NAME_LATN,GDP_ALT_MIL_USD_2017,YEAR
0,RS222,Braničevska oblast,279103.387230,2014
1,RS223,Zaječarska oblast,136427.451648,2014
2,RS224,Jablanička oblast,162346.301510,2014
3,RS221,Borska oblast,192494.141487,2014
4,RS214,Moravička oblast,323908.742375,2014


In [ ]:
path_list = glob.glob('/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/luminosity_bm_eu/*.tif')
path_list.sort()

In [ ]:
print(find_index(path_list, '2014'))
print(find_index(path_list, '2022'))

print(path_list[find_index(path_list, '2014')].find('2014'))

0
96
82


In [ ]:
df_list = list()
sel_paths = path_list

for path in tqdm(sel_paths):
    
    df_poly = poly.copy(deep=True)

    df_poly[f'luminosity'] = pd.DataFrame(zonal_stats(poly.geometry, path, stats='sum'))
    df_poly['ym'] = path[82:90]

    df_list.append(df_poly)

    del df_poly

gc.collect()

df_lumi = pd.concat(df_list, axis=0).reset_index(drop=True).drop(['geometry'], axis=1)
df_lumi['year'] = df_lumi['ym'].str[:4]

# if level == 1:

#     df_lumi = df_lumi.groupby(['GID_1', 'NAME_1', 'year']).agg({'luminosity': 'sum'}).reset_index()

# if level == 2:
#     df_lumi = df_lumi.groupby(['GID_2', 'NAME_2', 'GID_1', 'NAME_1', 'year']).agg({'luminosity': 'sum'}).reset_index()

df_lumi = df_lumi.groupby(['NUTS_ID', 'NAME_LATN', 'year']).agg({'luminosity': 'sum'}).reset_index()

df_lumi


  0%|          | 0/108 [00:00<?, ?it/s]

,NUTS_ID,NAME_LATN,year,luminosity
0,RS110,City of Belgrade,2014,1.546429e+06
1,RS110,City of Belgrade,2015,1.595494e+06
2,RS110,City of Belgrade,2016,1.619582e+06
3,RS110,City of Belgrade,2017,1.740217e+06
4,RS110,City of Belgrade,2018,1.722659e+06
...,...,...,...,...
220,RS229,Toplička oblast,2018,6.353792e+04
221,RS229,Toplička oblast,2019,7.078095e+04
222,RS229,Toplička oblast,2020,7.183000e+04
223,RS229,Toplička oblast,2021,8.044740e+04


In [ ]:
path_list = glob.glob('/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/population/*.tif')
path_list.sort()
path_list = path_list
path_list

['/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/population/srb_ppp_2014_1km_Aggregated.tif',
 '/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/population/srb_ppp_2015_1km_Aggregated.tif',
 '/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/population/srb_ppp_2016_1km_Aggregated.tif',
 '/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/population/srb_ppp_2017_1km_Aggregated.tif',
 '/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/population/srb_ppp_2018_1km_Aggregated.tif',
 '/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/population/srb_ppp_2019_1km_Aggregated.tif',
 '/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/population/srb_ppp_2020_1km_Aggregated.tif']

In [ ]:
print(path_list[find_index(path_list, '2014')].find('2014'))

77


In [ ]:
df_list = list()

for path in tqdm(path_list):
    
    df_poly = poly.copy(deep=True)

    df_poly[f'population'] = pd.DataFrame(zonal_stats(poly.geometry, path, stats='sum'))
    df_poly['year'] = path[77:81]

    df_list.append(df_poly)

    del df_poly

gc.collect()

df_pop = pd.concat(df_list, axis=0).reset_index(drop=True).drop(['geometry'], axis=1)
df_pop['population'] = df_pop['population']+1
df_pop

  0%|          | 0/7 [00:00<?, ?it/s]

,NUTS_ID,NAME_LATN,population,year
0,RS222,Braničevska oblast,178570.203125,2014
1,RS223,Zaječarska oblast,114605.054688,2014
2,RS224,Jablanička oblast,208821.078125,2014
3,RS221,Borska oblast,118276.703125,2014
4,RS214,Moravička oblast,208511.984375,2014
...,...,...,...,...
170,RS126,Srednjobanatska oblast,169848.843750,2020
171,RS127,Sremska oblast,289837.156250,2020
172,RS211,Zlatiborska oblast,260719.609375,2020
173,RS212,Kolubarska oblast,158942.718750,2020


In [ ]:
pop_2021 = df_pop[df_pop['year'] == '2020']
pop_2021['year'] = '2021'
pop_2021['population'] = pop_2021['population']*0.9917

pop_2022 = pop_2021.copy(deep=True)
pop_2022['year'] = '2022'
pop_2022['population'] = pop_2022['population']*0.9897

df_pop = pd.concat([df_pop, pop_2021, pop_2022], axis=0).reset_index(drop=True)
df_pop

/var/folders/cv/qgwv39c13dqgzq234dyzzxm80000gn/T/ipykernel_67789/1563104009.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_2021['year'] = '2021'
/var/folders/cv/qgwv39c13dqgzq234dyzzxm80000gn/T/ipykernel_67789/1563104009.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_2021['population'] = pop_2021['population']*0.9917


,NUTS_ID,NAME_LATN,population,year
0,RS222,Braničevska oblast,178570.203125,2014
1,RS223,Zaječarska oblast,114605.054688,2014
2,RS224,Jablanička oblast,208821.078125,2014
3,RS221,Borska oblast,118276.703125,2014
4,RS214,Moravička oblast,208511.984375,2014
...,...,...,...,...
220,RS126,Srednjobanatska oblast,166704.175634,2022
221,RS127,Sremska oblast,284470.963322,2022
222,RS211,Zlatiborska oblast,255892.513560,2022
223,RS212,Kolubarska oblast,155999.972194,2022


In [ ]:
path_list = glob.glob('/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/land_cover_eu/*.tif')
path_list.sort()
path_list = path_list
path_list

['/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/land_cover_eu/SRB_2016.tif',
 '/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/land_cover_eu/SRB_2017.tif',
 '/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/land_cover_eu/SRB_2018.tif',
 '/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/land_cover_eu/SRB_2019.tif',
 '/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/land_cover_eu/SRB_2020.tif',
 '/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/land_cover_eu/SRB_2021.tif',
 '/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/land_cover_eu/SRB_2022.tif']

In [ ]:
cmap = {0: 'water', 1: 'trees', 2: 'grass', 3: 'flooded_vegetation', 4: 'crops', 5: 'shrub_and_scrub', 
        6: 'built', 7: 'bare', 9: 'snow_and_ice'}
crs = 'EPSG:4326'

poly_lst = list()
    
for path in tqdm(path_list):

    ind = path.rfind('_')

    df_poly = poly.copy(deep=True)
    df_poly = df_poly.to_crs(crs)
    mean_tree = zonal_stats(df_poly.geometry, 
                            path,
                            categorical=True, category_map=cmap)
        
    df_poly['year'] = path[ind+1:ind+5]

    df = pd.DataFrame(mean_tree).fillna(0)
    # df['green_pct'] = 100*(df['trees']+df['grass']+df['shrub_and_scrub']) / df.sum(axis=1)
    df['built_pct'] = 100*df['built']/df.sum(axis=1)
    # df_poly = pd.concat([df_poly, df[['green_pct', 'built_pct']]], axis=1)
    df_poly = pd.concat([df_poly, df[['built_pct']]], axis=1)

    poly_lst.append(df_poly)
    
df_land_cover = pd.concat(poly_lst, axis=0).drop(['geometry'], axis=1)
df_land_cover = df_land_cover.sort_values(by=['NUTS_ID', 'NAME_LATN', 'year']).reset_index(drop=True)
# df_land_cover['green_cover_growth'] = df_land_cover.groupby(['NUTS_ID', 'NAME_LATN'])['green_pct'].pct_change() * 100
df_land_cover['built_cover_growth'] = df_land_cover.groupby(['NUTS_ID', 'NAME_LATN'])['built_pct'].pct_change() * 100

df_land_cover

  0%|          | 0/7 [00:00<?, ?it/s]

,NUTS_ID,NAME_LATN,year,built_pct,built_cover_growth
0,RS110,City of Belgrade,2016,15.543167,NaN
1,RS110,City of Belgrade,2017,16.344243,5.153881
2,RS110,City of Belgrade,2018,15.778829,-3.459407
3,RS110,City of Belgrade,2019,16.017129,1.510254
4,RS110,City of Belgrade,2020,16.156065,0.867417
...,...,...,...,...,...
170,RS229,Toplička oblast,2018,2.400710,-0.271003
171,RS229,Toplička oblast,2019,2.445071,1.847826
172,RS229,Toplička oblast,2020,2.486822,1.707577
173,RS229,Toplička oblast,2021,2.651219,6.610703


In [ ]:
def get_pca_comp(df, variables):
    X = df[variables]

    # Scale the data to have a mean of 0 and a standard deviation of 1
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Use PCA to perform dimensionality reduction
    pca = PCA()
    pca.fit(X_scaled)   

    # Determine how many principal components to keep based on the explained variance ratio
    explained_variance_ratio = pca.explained_variance_ratio_
    n_components = len([evr for evr in explained_variance_ratio if evr >= 0.05])

    # Fit PCA with the chosen number of components
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X_scaled)

    # Calculate the principal component scores for each observation in the original data
    scores = pd.DataFrame(X_pca, columns=['PC{}'.format(i+1) for i in range(n_components)])
    scores.index = df.index

    # Combine the principal component scores into a single index by assigning weights
    # based on the explained variance ratio of each component
    weights = explained_variance_ratio[:n_components]
    index = scores.dot(weights)

    return index

In [ ]:
df_lumi

,NUTS_ID,NAME_LATN,year,luminosity
0,RS110,City of Belgrade,2014,1.546429e+06
1,RS110,City of Belgrade,2015,1.595494e+06
2,RS110,City of Belgrade,2016,1.619582e+06
3,RS110,City of Belgrade,2017,1.740217e+06
4,RS110,City of Belgrade,2018,1.722659e+06
...,...,...,...,...
220,RS229,Toplička oblast,2018,6.353792e+04
221,RS229,Toplička oblast,2019,7.078095e+04
222,RS229,Toplička oblast,2020,7.183000e+04
223,RS229,Toplička oblast,2021,8.044740e+04


In [ ]:
# if level == 1:
#     df_econ = pd.merge(df_lumi, df_pop, on=['GID_1', 'NAME_1', 'year'], how='left')
# if level == 2:
#     df_econ = pd.merge(df_lumi, df_pop, on=['GID_2', 'NAME_2', 'GID_1', 'NAME_1', 'year'], how='left')
# df_econ['lpc'] = df_econ['luminosity'] / df_econ['population']

# df_econ = df_econ.sort_values(by=[f'GID_{level}', 'year'], ascending=True)
# df_econ['luminosity_pct_change'] = 100*(df_econ.groupby(f'GID_{level}')['luminosity'].apply(pd.Series.pct_change))
# df_econ['lpc_pct_change'] = 100*(df_econ.groupby(f'GID_{level}')['lpc'].apply(pd.Series.pct_change))
# df_econ = df_econ[df_econ['year'] > '2014'].reset_index(drop=True)
# df_econ = df_econ.fillna(0)
# df_econ.head()

In [ ]:
df_econ = pd.merge(df_lumi, df_pop, on=['NUTS_ID', 'NAME_LATN', 'year'], how='left')
df_econ = pd.merge(df_econ, df_land_cover, on=['NUTS_ID', 'NAME_LATN', 'year'], how='left')
df_econ['lpc'] = df_econ['luminosity'] / df_econ['population']
df_econ = df_econ.sort_values(by=['NUTS_ID', 'NAME_LATN', 'year'], ascending=True)
df_econ['luminosity_pct_change'] = 100*(df_econ.groupby('NUTS_ID')['luminosity'].apply(pd.Series.pct_change))
df_econ['lpc_pct_change'] = 100*(df_econ.groupby(f'NUTS_ID')['lpc'].apply(pd.Series.pct_change))
df_econ = df_econ[df_econ['year'] > '2016'].reset_index(drop=True)

df_econ.head()

,NUTS_ID,NAME_LATN,year,luminosity,population,built_pct,built_cover_growth,lpc,luminosity_pct_change,lpc_pct_change
0,RS110,City of Belgrade,2017,1.740217e+06,1721511.250,16.344243,5.153881,1.010866,7.448528,6.784118
1,RS110,City of Belgrade,2018,1.722659e+06,1732401.375,15.778829,-3.459407,0.994376,-1.008951,-1.631223
2,RS110,City of Belgrade,2019,1.802029e+06,1743370.500,16.017129,1.510254,1.033647,4.607440,3.949260
3,RS110,City of Belgrade,2020,1.770475e+06,1754560.000,16.156065,0.867417,1.009070,-1.751067,-2.377638
4,RS110,City of Belgrade,2021,1.943496e+06,1739997.152,16.587819,2.672400,1.116954,9.772625,10.691364


In [ ]:
vars_1 = ['lpc', 'lpc_pct_change', 'built_pct', 'built_cover_growth']
vars_2 = ['luminosity', 'luminosity_pct_change']
vars_3 = ['lpc', 'luminosity_pct_change']

df_econ['econ_score_og'] = get_pca_comp(df_econ, vars_1)
df_econ['econ_score_og'] = df_econ.groupby(['year'])['econ_score_og'].rank(pct=True, ascending=True)*100

df_econ['econ_score_alt_1'] = get_pca_comp(df_econ, vars_2)
df_econ['econ_score_alt_1'] = df_econ.groupby(['year'])['econ_score_alt_1'].rank(pct=True, ascending=True)*100

df_econ['econ_score_alt_2'] = get_pca_comp(df_econ, vars_3)
df_econ['econ_score_alt_2'] = df_econ.groupby(['year'])['econ_score_alt_2'].rank(pct=True, ascending=True)*100

df_econ.columns = df_econ.columns.str.upper()
df_econ['YEAR'] = df_econ['YEAR'].astype(int)
df_econ.head()

,NUTS_ID,NAME_LATN,YEAR,LUMINOSITY,POPULATION,BUILT_PCT,BUILT_COVER_GROWTH,LPC,LUMINOSITY_PCT_CHANGE,LPC_PCT_CHANGE,ECON_SCORE_OG,ECON_SCORE_ALT_1,ECON_SCORE_ALT_2
0,RS110,City of Belgrade,2017,1.740217e+06,1721511.250,16.344243,5.153881,1.010866,7.448528,6.784118,52.0,100.0,44.0
1,RS110,City of Belgrade,2018,1.722659e+06,1732401.375,15.778829,-3.459407,0.994376,-1.008951,-1.631223,80.0,100.0,28.0
2,RS110,City of Belgrade,2019,1.802029e+06,1743370.500,16.017129,1.510254,1.033647,4.607440,3.949260,68.0,100.0,64.0
3,RS110,City of Belgrade,2020,1.770475e+06,1754560.000,16.156065,0.867417,1.009070,-1.751067,-2.377638,52.0,100.0,84.0
4,RS110,City of Belgrade,2021,1.943496e+06,1739997.152,16.587819,2.672400,1.116954,9.772625,10.691364,48.0,100.0,48.0


In [ ]:
eu_gdp_ppp = pd.read_csv('EU_GDP_2021_PPP.csv')
eu_gdppc_ppp = pd.read_csv('EU_GDP_PER_CAPITA_2021_PPP.csv')

In [ ]:
eu_gdp_ppp.columns

Index(['VERSIONS', 'LEVEL_ID', 'TERRITORY_ID', 'NAME_HTML', 'YEAR', 'DATE',
       'UNIT', 'VALUE'],
      dtype='object')

In [ ]:
eu_gdppc_ppp.columns

Index(['VERSIONS', 'LEVEL_ID', 'TERRITORY_ID', 'NAME_HTML', 'YEAR', 'DATE',
       'UNIT', 'VALUE'],
      dtype='object')

In [ ]:
eu_gdp_ppp = eu_gdp_ppp[(eu_gdp_ppp['TERRITORY_ID'].str.contains('RS')) \
                    & (eu_gdp_ppp['LEVEL_ID'] == 3) \
                    & (eu_gdp_ppp['UNIT'] == 'Million EUR')]

eu_gdp_ppp = eu_gdp_ppp.drop(['VERSIONS', 'LEVEL_ID', 'NAME_HTML', 'DATE', 'UNIT'], axis=1)
eu_gdp_ppp = eu_gdp_ppp.rename({'TERRITORY_ID': 'NUTS_ID', 'VALUE': 'GDP_EU_MIL_EUR_2021'}, axis=1)
eu_gdp_ppp

,NUTS_ID,YEAR,GDP_EU_MIL_EUR_2021
75613,RS110,2012,13371.53
75614,RS110,2013,14534.25
75615,RS110,2014,13740.95
75616,RS110,2015,14226.53
75617,RS110,2016,14772.14
...,...,...,...
76024,RSZZZ,2020,21.55
76025,RSZZZ,2021,23.44
76026,RSZZZ,2022,26.53
76027,RSZZZ,2023,30.51


In [ ]:
eu_gdppc_ppp = eu_gdppc_ppp[(eu_gdppc_ppp['TERRITORY_ID'].str.contains('RS')) \
                    & (eu_gdppc_ppp['LEVEL_ID'] == 3) \
                    & (eu_gdppc_ppp['UNIT'] == 'EUR')]

eu_gdppc_ppp = eu_gdppc_ppp.drop(['VERSIONS', 'LEVEL_ID', 'NAME_HTML', 'DATE', 'UNIT'], axis=1)
eu_gdppc_ppp = eu_gdppc_ppp.rename({'TERRITORY_ID': 'NUTS_ID', 'VALUE': 'GDPPC_EU_EUR_2021'}, axis=1)
eu_gdppc_ppp

,NUTS_ID,YEAR,GDPPC_EU_EUR_2021
73621,RS110,2012,8034.71
73622,RS110,2013,8705.49
73623,RS110,2014,8203.36
73624,RS110,2015,8468.68
73625,RS110,2016,8772.26
...,...,...,...
73993,RS229,2020,3689.66
73994,RS229,2021,4675.29
73995,RS229,2022,5342.84
73996,RS229,2023,6182.61


In [ ]:
# if level == 1:
#     df_econ = pd.merge(df_econ, df_gdp_ppp, on=['GID_1', 'NAME_1', 'year'], how='left')
#     df_econ = pd.merge(df_econ, df_gdp_alt, on=['GID_1', 'NAME_1', 'year'], how='left')

# if level == 2:
#     df_econ = pd.merge(df_econ, df_gdp_ppp, on=['GID_2', 'NAME_2', 'GID_1', 'NAME_1', 'year'], how='left')
#     df_econ = pd.merge(df_econ, df_gdp_alt, on=['GID_2', 'NAME_2', 'GID_1', 'NAME_1', 'year'], how='left')
df_econ = pd.merge(df_econ, df_gdp_alt, on=['NUTS_ID', 'NAME_LATN', 'YEAR'], how='left')
df_econ = pd.merge(df_econ, eu_gdp_ppp, on=['NUTS_ID', 'YEAR'], how='left')
df_econ = pd.merge(df_econ, eu_gdppc_ppp, on=['NUTS_ID', 'YEAR'], how='left')

df_econ.head()

,NUTS_ID,NAME_LATN,YEAR,LUMINOSITY,POPULATION,BUILT_PCT,BUILT_COVER_GROWTH,LPC,LUMINOSITY_PCT_CHANGE,LPC_PCT_CHANGE,ECON_SCORE_OG,ECON_SCORE_ALT_1,ECON_SCORE_ALT_2,GDP_ALT_MIL_USD_2017,GDP_EU_MIL_EUR_2021,GDPPC_EU_EUR_2021
0,RS110,City of Belgrade,2017,1.740217e+06,1721511.250,16.344243,5.153881,1.010866,7.448528,6.784118,52.0,100.0,44.0,2.210398e+06,15853.22,9396.56
1,RS110,City of Belgrade,2018,1.722659e+06,1732401.375,15.778829,-3.459407,0.994376,-1.008951,-1.631223,80.0,100.0,28.0,2.303726e+06,17735.51,10493.20
2,RS110,City of Belgrade,2019,1.802029e+06,1743370.500,16.017129,1.510254,1.033647,4.607440,3.949260,68.0,100.0,64.0,2.396335e+06,19361.94,11429.31
3,RS110,City of Belgrade,2020,1.770475e+06,1754560.000,16.156065,0.867417,1.009070,-1.751067,-2.377638,52.0,100.0,84.0,NaN,19717.11,11636.08
4,RS110,City of Belgrade,2021,1.943496e+06,1739997.152,16.587819,2.672400,1.116954,9.772625,10.691364,48.0,100.0,48.0,NaN,21422.69,12686.13


In [ ]:
df_econ

,NUTS_ID,NAME_LATN,YEAR,LUMINOSITY,POPULATION,BUILT_PCT,BUILT_COVER_GROWTH,LPC,LUMINOSITY_PCT_CHANGE,LPC_PCT_CHANGE,ECON_SCORE_OG,ECON_SCORE_ALT_1,ECON_SCORE_ALT_2,GDP_ALT_MIL_USD_2017,GDP_EU_MIL_EUR_2021,GDPPC_EU_EUR_2021
0,RS110,City of Belgrade,2017,1.740217e+06,1.721511e+06,16.344243,5.153881,1.010866,7.448528,6.784118,52.0,100.0,44.0,2.210398e+06,15853.22,9396.56
1,RS110,City of Belgrade,2018,1.722659e+06,1.732401e+06,15.778829,-3.459407,0.994376,-1.008951,-1.631223,80.0,100.0,28.0,2.303726e+06,17735.51,10493.20
2,RS110,City of Belgrade,2019,1.802029e+06,1.743370e+06,16.017129,1.510254,1.033647,4.607440,3.949260,68.0,100.0,64.0,2.396335e+06,19361.94,11429.31
3,RS110,City of Belgrade,2020,1.770475e+06,1.754560e+06,16.156065,0.867417,1.009070,-1.751067,-2.377638,52.0,100.0,84.0,NaN,19717.11,11636.08
4,RS110,City of Belgrade,2021,1.943496e+06,1.739997e+06,16.587819,2.672400,1.116954,9.772625,10.691364,48.0,100.0,48.0,NaN,21422.69,12686.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,RS229,Toplička oblast,2018,6.353792e+04,8.466139e+04,2.400710,-0.271003,0.750495,0.567617,1.770736,92.0,4.0,20.0,1.081189e+05,303.51,3602.49
146,RS229,Toplička oblast,2019,7.078095e+04,8.367711e+04,2.445071,1.847826,0.845882,11.399545,12.709921,80.0,8.0,80.0,1.126029e+05,323.94,3893.51
147,RS229,Toplička oblast,2020,7.183000e+04,8.268509e+04,2.486822,1.707577,0.868718,1.482108,2.699641,68.0,12.0,92.0,NaN,302.81,3689.66
148,RS229,Toplička oblast,2021,8.044740e+04,8.199881e+04,2.651219,6.610703,0.981080,11.996937,12.934292,72.0,20.0,56.0,NaN,378.17,4675.29


In [ ]:
df_econ.to_csv('srb_eu_validation.csv', index=False)

In [ ]:
path_list = glob.glob('/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/population_alt/*.tif')
path_list.sort()
path_list = path_list
path_list

['/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/population_alt/population_SRB_2016.tif',
 '/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/population_alt/population_SRB_2017.tif',
 '/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/population_alt/population_SRB_2018.tif',
 '/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/population_alt/population_SRB_2019.tif',
 '/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/population_alt/population_SRB_2020.tif',
 '/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/population_alt/population_SRB_2021.tif',
 '/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/population_alt/population_SRB_2022.tif']

In [ ]:
def_affine = rasterio.open(path_list[1]).profile['transform']
def_affine

Affine(0.004491576420597608, 0.0, 18.855637813668757,
       0.0, -0.004491576420597608, 46.1913719094258)

In [ ]:
print(path_list[find_index(path_list, '2018')].find('2018'))

88


In [ ]:
df_list = list()

for path in tqdm(path_list):
    
    df_poly = poly.copy(deep=True)

    df_poly[f'POPULATION_ALT'] = pd.DataFrame(zonal_stats(poly.geometry, rasterio.open(path).read(1), affine=def_affine, stats='sum'))
    df_poly['YEAR'] = path[88:92]

    df_list.append(df_poly)

    del df_poly

gc.collect()

df_pop = pd.concat(df_list, axis=0).reset_index(drop=True).drop(['geometry'], axis=1)
# df_pop['population'] = df_pop['population']+1
df_pop['YEAR'] = df_pop['YEAR'].astype(int)
df_pop

  0%|          | 0/7 [00:00<?, ?it/s]

,NUTS_ID,NAME_LATN,POPULATION_ALT,YEAR
0,RS222,Braničevska oblast,793137.0,2016
1,RS223,Zaječarska oblast,112092.0,2016
2,RS224,Jablanička oblast,773759.0,2016
3,RS221,Borska oblast,549495.0,2016
4,RS214,Moravička oblast,2180165.0,2016
...,...,...,...,...
170,RS126,Srednjobanatska oblast,591284.0,2022
171,RS127,Sremska oblast,999719.0,2022
172,RS211,Zlatiborska oblast,891877.0,2022
173,RS212,Kolubarska oblast,552823.0,2022


In [ ]:
path_list = glob.glob('/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/luminosity_bm_eu/*.tif')
path_list.sort()

In [ ]:
path_list

['/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/luminosity_bm_eu/SRB_bm_2014_01.tif',
 '/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/luminosity_bm_eu/SRB_bm_2014_02.tif',
 '/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/luminosity_bm_eu/SRB_bm_2014_03.tif',
 '/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/luminosity_bm_eu/SRB_bm_2014_04.tif',
 '/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/luminosity_bm_eu/SRB_bm_2014_05.tif',
 '/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/luminosity_bm_eu/SRB_bm_2014_06.tif',
 '/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/luminosity_bm_eu/SRB_bm_2014_07.tif',
 '/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/luminosity_bm_eu/SRB_bm_2014_08.tif',
 '/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/luminosity_bm_eu/SRB_bm_2014_09.tif',
 '/Users/sonle/Documents/work/WB/wb_pimpam_ged/SRB/raw_data/luminosity_bm_eu/SRB_bm_2014_10.tif',
 '/Users/sonle/Docum

In [ ]:
print(find_index(path_list, '2014'))
print(find_index(path_list, '2022'))

print(path_list[find_index(path_list, '2014')].find('2014'))

0
96
82


In [ ]:
df_list = list()
sel_paths = path_list

for path in tqdm(sel_paths):
    
    df_poly = poly.copy(deep=True)

    df_poly[f'LUMINOSITY_ALT'] = pd.DataFrame(zonal_stats(poly.geometry, path, stats='sum'))
    df_poly['ym'] = path[82:86]

    df_list.append(df_poly)

    del df_poly

gc.collect()

df_lumi = pd.concat(df_list, axis=0).reset_index(drop=True).drop(['geometry'], axis=1)
df_lumi['YEAR'] = df_lumi['ym'].str[:4]

df_lumi = df_lumi.groupby(['NUTS_ID', 'NAME_LATN', 'YEAR']).agg({'LUMINOSITY_ALT': 'sum'}).reset_index()

# if level == 1:

#     df_lumi = df_lumi.groupby(['GID_1', 'NAME_1', 'year']).agg({'luminosity': 'sum'}).reset_index()

# if level == 2:
#     df_lumi = df_lumi.groupby(['GID_2', 'NAME_2', 'GID_1', 'NAME_1', 'year']).agg({'luminosity': 'sum'}).reset_index()

df_lumi['YEAR'] = df_lumi['YEAR'].astype(int)
df_lumi = df_lumi[df_lumi['YEAR'] > 2015].reset_index(drop=True)
df_lumi


  0%|          | 0/108 [00:00<?, ?it/s]

,NUTS_ID,NAME_LATN,YEAR,LUMINOSITY_ALT
0,RS110,City of Belgrade,2016,1.619582e+06
1,RS110,City of Belgrade,2017,1.740217e+06
2,RS110,City of Belgrade,2018,1.722659e+06
3,RS110,City of Belgrade,2019,1.802029e+06
4,RS110,City of Belgrade,2020,1.770475e+06
...,...,...,...,...
170,RS229,Toplička oblast,2018,6.353792e+04
171,RS229,Toplička oblast,2019,7.078095e+04
172,RS229,Toplička oblast,2020,7.183000e+04
173,RS229,Toplička oblast,2021,8.044740e+04


In [ ]:
df_lumi = pd.merge(df_lumi, df_pop, on=['NUTS_ID', 'NAME_LATN', 'YEAR'], how='left')
df_lumi['LPC_ALT'] = df_lumi['LUMINOSITY_ALT'] / df_lumi['POPULATION_ALT']
df_lumi = df_lumi.sort_values(by=['NUTS_ID', 'NAME_LATN', 'YEAR'], ascending=True)
df_lumi['LPC_ALT_PCT_CHANGE'] = 100*(df_lumi.groupby(f'NUTS_ID')['LPC_ALT'].apply(pd.Series.pct_change))
df_lumi

,NUTS_ID,NAME_LATN,YEAR,LUMINOSITY_ALT,POPULATION_ALT,LPC_ALT,LPC_ALT_PCT_CHANGE
0,RS110,City of Belgrade,2016,1.619582e+06,1114335.0,1.453407,NaN
1,RS110,City of Belgrade,2017,1.740217e+06,5805150.0,0.299771,-79.374580
2,RS110,City of Belgrade,2018,1.722659e+06,1099508.0,1.566754,422.650028
3,RS110,City of Belgrade,2019,1.802029e+06,5836864.0,0.308732,-80.294775
4,RS110,City of Belgrade,2020,1.770475e+06,1090970.0,1.622844,425.647509
...,...,...,...,...,...,...,...
170,RS229,Toplička oblast,2018,6.353792e+04,360762.0,0.176121,-13.600864
171,RS229,Toplička oblast,2019,7.078095e+04,301571.0,0.234707,33.264547
172,RS229,Toplička oblast,2020,7.183000e+04,358660.0,0.200273,-14.671107
173,RS229,Toplička oblast,2021,8.044740e+04,356897.0,0.225408,12.550180


In [ ]:
df_econ = pd.read_csv('/Users/sonle/Documents/work/WB/wb_pimpam_ged/validation/econ/srb_eu_validation.csv')
df_econ.head()

,NUTS_ID,NAME_LATN,YEAR,LUMINOSITY,POPULATION,BUILT_PCT,BUILT_COVER_GROWTH,LPC,LUMINOSITY_PCT_CHANGE,LPC_PCT_CHANGE,ECON_SCORE_OG,ECON_SCORE_ALT_1,ECON_SCORE_ALT_2,GDP_ALT_MIL_USD_2017,GDP_EU_MIL_EUR_2021,GDPPC_EU_EUR_2021
0,RS110,City of Belgrade,2017,1.740217e+06,1721511.250,16.344243,5.153881,1.010866,7.448528,6.784118,52.0,100.0,44.0,2.210398e+06,15853.22,9396.56
1,RS110,City of Belgrade,2018,1.722659e+06,1732401.375,15.778829,-3.459407,0.994376,-1.008951,-1.631223,80.0,100.0,28.0,2.303726e+06,17735.51,10493.20
2,RS110,City of Belgrade,2019,1.802029e+06,1743370.500,16.017129,1.510254,1.033647,4.607440,3.949260,68.0,100.0,64.0,2.396335e+06,19361.94,11429.31
3,RS110,City of Belgrade,2020,1.770475e+06,1754560.000,16.156065,0.867417,1.009070,-1.751067,-2.377638,52.0,100.0,84.0,NaN,19717.11,11636.08
4,RS110,City of Belgrade,2021,1.943496e+06,1739997.152,16.587819,2.672400,1.116954,9.772625,10.691364,48.0,100.0,48.0,NaN,21422.69,12686.13


In [ ]:
df_econ = pd.merge(df_econ, df_lumi.drop(['LUMINOSITY_ALT'], axis=1), on=['NUTS_ID', 'NAME_LATN', 'YEAR'], how='left')

# df_econ['LPC_ALT'] = df_econ['LUMINOSITY'] / df_econ['POPULATION_ALT']
# df_econ = df_econ.sort_values(by=['NUTS_ID', 'NAME_LATN', 'YEAR'], ascending=True)
# df_econ['LPC_ALT_PCT_CHANGE'] = 100*(df_econ.groupby(f'NUTS_ID')['LPC_ALT'].apply(pd.Series.pct_change))

df_econ

,NUTS_ID,NAME_LATN,YEAR,LUMINOSITY,POPULATION,BUILT_PCT,BUILT_COVER_GROWTH,LPC,LUMINOSITY_PCT_CHANGE,LPC_PCT_CHANGE,ECON_SCORE_OG,ECON_SCORE_ALT_1,ECON_SCORE_ALT_2,GDP_ALT_MIL_USD_2017,GDP_EU_MIL_EUR_2021,GDPPC_EU_EUR_2021,POPULATION_ALT,LPC_ALT,LPC_ALT_PCT_CHANGE
0,RS110,City of Belgrade,2017,1.740217e+06,1.721511e+06,16.344243,5.153881,1.010866,7.448528,6.784118,52.0,100.0,44.0,2.210398e+06,15853.22,9396.56,5805150.0,0.299771,-79.374580
1,RS110,City of Belgrade,2018,1.722659e+06,1.732401e+06,15.778829,-3.459407,0.994376,-1.008951,-1.631223,80.0,100.0,28.0,2.303726e+06,17735.51,10493.20,1099508.0,1.566754,422.650028
2,RS110,City of Belgrade,2019,1.802029e+06,1.743370e+06,16.017129,1.510254,1.033647,4.607440,3.949260,68.0,100.0,64.0,2.396335e+06,19361.94,11429.31,5836864.0,0.308732,-80.294775
3,RS110,City of Belgrade,2020,1.770475e+06,1.754560e+06,16.156065,0.867417,1.009070,-1.751067,-2.377638,52.0,100.0,84.0,NaN,19717.11,11636.08,1090970.0,1.622844,425.647509
4,RS110,City of Belgrade,2021,1.943496e+06,1.739997e+06,16.587819,2.672400,1.116954,9.772625,10.691364,48.0,100.0,48.0,NaN,21422.69,12686.13,1082784.0,1.794907,10.602522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,RS229,Toplička oblast,2018,6.353792e+04,8.466139e+04,2.400710,-0.271003,0.750495,0.567617,1.770736,92.0,4.0,20.0,1.081189e+05,303.51,3602.49,360762.0,0.176121,-13.600864
146,RS229,Toplička oblast,2019,7.078095e+04,8.367711e+04,2.445071,1.847826,0.845882,11.399545,12.709921,80.0,8.0,80.0,1.126029e+05,323.94,3893.51,301571.0,0.234707,33.264547
147,RS229,Toplička oblast,2020,7.183000e+04,8.268509e+04,2.486822,1.707577,0.868718,1.482108,2.699641,68.0,12.0,92.0,NaN,302.81,3689.66,358660.0,0.200273,-14.671107
148,RS229,Toplička oblast,2021,8.044740e+04,8.199881e+04,2.651219,6.610703,0.981080,11.996937,12.934292,72.0,20.0,56.0,NaN,378.17,4675.29,356897.0,0.225408,12.550180


In [ ]:
vars_1 = ['LUMINOSITY', 'LUMINOSITY_PCT_CHANGE', 'BUILT_PCT', 'BUILT_COVER_GROWTH']
vars_2 = ['LUMINOSITY', 'BUILT_PCT']


df_econ['ECON_SCORE_ALT_3'] = get_pca_comp(df_econ, vars_1)
df_econ['ECON_SCORE_ALT_3'] = df_econ.groupby(['YEAR'])['ECON_SCORE_ALT_3'].rank(pct=True, ascending=True)*100

df_econ['ECON_SCORE_ALT_4'] = get_pca_comp(df_econ, vars_2)
df_econ['ECON_SCORE_ALT_4'] = df_econ.groupby(['YEAR'])['ECON_SCORE_ALT_4'].rank(pct=True, ascending=True)*100


In [ ]:
df_econ

,NUTS_ID,NAME_LATN,YEAR,LUMINOSITY,POPULATION,BUILT_PCT,BUILT_COVER_GROWTH,LPC,LUMINOSITY_PCT_CHANGE,LPC_PCT_CHANGE,ECON_SCORE_OG,ECON_SCORE_ALT_1,ECON_SCORE_ALT_2,GDP_ALT_MIL_USD_2017,GDP_EU_MIL_EUR_2021,GDPPC_EU_EUR_2021,ECON_SCORE_ALT_3,ECON_SCORE_ALT_4
0,RS110,City of Belgrade,2017,1.740217e+06,1.721511e+06,16.344243,5.153881,1.010866,7.448528,6.784118,52.0,100.0,44.0,2.210398e+06,15853.22,9396.56,100.0,100.0
1,RS110,City of Belgrade,2018,1.722659e+06,1.732401e+06,15.778829,-3.459407,0.994376,-1.008951,-1.631223,80.0,100.0,28.0,2.303726e+06,17735.51,10493.20,100.0,100.0
2,RS110,City of Belgrade,2019,1.802029e+06,1.743370e+06,16.017129,1.510254,1.033647,4.607440,3.949260,68.0,100.0,64.0,2.396335e+06,19361.94,11429.31,100.0,100.0
3,RS110,City of Belgrade,2020,1.770475e+06,1.754560e+06,16.156065,0.867417,1.009070,-1.751067,-2.377638,52.0,100.0,84.0,NaN,19717.11,11636.08,100.0,100.0
4,RS110,City of Belgrade,2021,1.943496e+06,1.739997e+06,16.587819,2.672400,1.116954,9.772625,10.691364,48.0,100.0,48.0,NaN,21422.69,12686.13,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,RS229,Toplička oblast,2018,6.353792e+04,8.466139e+04,2.400710,-0.271003,0.750495,0.567617,1.770736,92.0,4.0,20.0,1.081189e+05,303.51,3602.49,36.0,12.0
146,RS229,Toplička oblast,2019,7.078095e+04,8.367711e+04,2.445071,1.847826,0.845882,11.399545,12.709921,80.0,8.0,80.0,1.126029e+05,323.94,3893.51,24.0,12.0
147,RS229,Toplička oblast,2020,7.183000e+04,8.268509e+04,2.486822,1.707577,0.868718,1.482108,2.699641,68.0,12.0,92.0,NaN,302.81,3689.66,44.0,12.0
148,RS229,Toplička oblast,2021,8.044740e+04,8.199881e+04,2.651219,6.610703,0.981080,11.996937,12.934292,72.0,20.0,56.0,NaN,378.17,4675.29,40.0,16.0


In [ ]:
df_econ.to_csv('srb_eu_validation_ver2.csv', index=False)

# Analysis

## GDP

In [4]:
df = pd.read_csv('srb_eu_validation_ver2.csv')
df

,NUTS_ID,NAME_LATN,YEAR,LUMINOSITY,POPULATION,BUILT_PCT,BUILT_COVER_GROWTH,LPC,LUMINOSITY_PCT_CHANGE,LPC_PCT_CHANGE,ECON_SCORE_OG,ECON_SCORE_ALT_1,ECON_SCORE_ALT_2,GDP_ALT_MIL_USD_2017,GDP_EU_MIL_EUR_2021,GDPPC_EU_EUR_2021,ECON_SCORE_ALT_3,ECON_SCORE_ALT_4
0,RS110,City of Belgrade,2017,1.740217e+06,1.721511e+06,16.344243,5.153881,1.010866,7.448528,6.784118,52.0,100.0,44.0,2.210398e+06,15853.22,9396.56,100.0,100.0
1,RS110,City of Belgrade,2018,1.722659e+06,1.732401e+06,15.778829,-3.459407,0.994376,-1.008951,-1.631223,80.0,100.0,28.0,2.303726e+06,17735.51,10493.20,100.0,100.0
2,RS110,City of Belgrade,2019,1.802029e+06,1.743370e+06,16.017129,1.510254,1.033647,4.607440,3.949260,68.0,100.0,64.0,2.396335e+06,19361.94,11429.31,100.0,100.0
3,RS110,City of Belgrade,2020,1.770475e+06,1.754560e+06,16.156065,0.867417,1.009070,-1.751067,-2.377638,52.0,100.0,84.0,NaN,19717.11,11636.08,100.0,100.0
4,RS110,City of Belgrade,2021,1.943496e+06,1.739997e+06,16.587819,2.672400,1.116954,9.772625,10.691364,48.0,100.0,48.0,NaN,21422.69,12686.13,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,RS229,Toplička oblast,2018,6.353792e+04,8.466139e+04,2.400710,-0.271003,0.750495,0.567617,1.770736,92.0,4.0,20.0,1.081189e+05,303.51,3602.49,36.0,12.0
146,RS229,Toplička oblast,2019,7.078095e+04,8.367711e+04,2.445071,1.847826,0.845882,11.399545,12.709921,80.0,8.0,80.0,1.126029e+05,323.94,3893.51,24.0,12.0
147,RS229,Toplička oblast,2020,7.183000e+04,8.268509e+04,2.486822,1.707577,0.868718,1.482108,2.699641,68.0,12.0,92.0,NaN,302.81,3689.66,44.0,12.0
148,RS229,Toplička oblast,2021,8.044740e+04,8.199881e+04,2.651219,6.610703,0.981080,11.996937,12.934292,72.0,20.0,56.0,NaN,378.17,4675.29,40.0,16.0


In [5]:
df['RANK_GDP_EU'] = df.groupby(['YEAR'])['GDP_EU_MIL_EUR_2021'].rank(pct=True, ascending=True)*100
df['RANK_GDPPC_EU'] = df.groupby(['YEAR'])['GDPPC_EU_EUR_2021'].rank(pct=True, ascending=True)*100

In [6]:
df

,NUTS_ID,NAME_LATN,YEAR,LUMINOSITY,POPULATION,BUILT_PCT,BUILT_COVER_GROWTH,LPC,LUMINOSITY_PCT_CHANGE,LPC_PCT_CHANGE,ECON_SCORE_OG,ECON_SCORE_ALT_1,ECON_SCORE_ALT_2,GDP_ALT_MIL_USD_2017,GDP_EU_MIL_EUR_2021,GDPPC_EU_EUR_2021,ECON_SCORE_ALT_3,ECON_SCORE_ALT_4,RANK_GDP_EU,RANK_GDPPC_EU
0,RS110,City of Belgrade,2017,1.740217e+06,1.721511e+06,16.344243,5.153881,1.010866,7.448528,6.784118,52.0,100.0,44.0,2.210398e+06,15853.22,9396.56,100.0,100.0,100.0,100.0
1,RS110,City of Belgrade,2018,1.722659e+06,1.732401e+06,15.778829,-3.459407,0.994376,-1.008951,-1.631223,80.0,100.0,28.0,2.303726e+06,17735.51,10493.20,100.0,100.0,100.0,100.0
2,RS110,City of Belgrade,2019,1.802029e+06,1.743370e+06,16.017129,1.510254,1.033647,4.607440,3.949260,68.0,100.0,64.0,2.396335e+06,19361.94,11429.31,100.0,100.0,100.0,100.0
3,RS110,City of Belgrade,2020,1.770475e+06,1.754560e+06,16.156065,0.867417,1.009070,-1.751067,-2.377638,52.0,100.0,84.0,NaN,19717.11,11636.08,100.0,100.0,100.0,100.0
4,RS110,City of Belgrade,2021,1.943496e+06,1.739997e+06,16.587819,2.672400,1.116954,9.772625,10.691364,48.0,100.0,48.0,NaN,21422.69,12686.13,100.0,100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,RS229,Toplička oblast,2018,6.353792e+04,8.466139e+04,2.400710,-0.271003,0.750495,0.567617,1.770736,92.0,4.0,20.0,1.081189e+05,303.51,3602.49,36.0,12.0,4.0,24.0
146,RS229,Toplička oblast,2019,7.078095e+04,8.367711e+04,2.445071,1.847826,0.845882,11.399545,12.709921,80.0,8.0,80.0,1.126029e+05,323.94,3893.51,24.0,12.0,4.0,24.0
147,RS229,Toplička oblast,2020,7.183000e+04,8.268509e+04,2.486822,1.707577,0.868718,1.482108,2.699641,68.0,12.0,92.0,NaN,302.81,3689.66,44.0,12.0,4.0,24.0
148,RS229,Toplička oblast,2021,8.044740e+04,8.199881e+04,2.651219,6.610703,0.981080,11.996937,12.934292,72.0,20.0,56.0,NaN,378.17,4675.29,40.0,16.0,4.0,24.0


In [64]:
for x in ['ECON_SCORE_OG', 'ECON_SCORE_ALT_1', 'ECON_SCORE_ALT_2', 'ECON_SCORE_ALT_3', 'ECON_SCORE_ALT_4']:
    print(x)    
    print(spearmanr(df[x], df['RANK_GDP_EU']))
    print('\n')

ECON_SCORE_OG
SignificanceResult(statistic=0.016923076923076923, pvalue=0.8371460533245417)


ECON_SCORE_ALT_1
SignificanceResult(statistic=0.8146153846153847, pvalue=7.783589304325791e-37)


ECON_SCORE_ALT_2
SignificanceResult(statistic=-0.009102564102564104, pvalue=0.9119709258869143)


ECON_SCORE_ALT_3
SignificanceResult(statistic=0.533974358974359, pvalue=1.969875228636274e-12)


ECON_SCORE_ALT_4
SignificanceResult(statistic=0.6683333333333333, pvalue=9.298872512533859e-21)




In [65]:
for x in ['ECON_SCORE_OG', 'ECON_SCORE_ALT_1', 'ECON_SCORE_ALT_2', 'ECON_SCORE_ALT_3', 'ECON_SCORE_ALT_4']:
    print(x)    
    print(np.cov(df[x], df['RANK_GDP_EU']))
    print('\n')

ECON_SCORE_OG
[[837.58389262  14.17449664]
 [ 14.17449664 837.58389262]]


ECON_SCORE_ALT_1
[[837.58389262 682.30872483]
 [682.30872483 837.58389262]]


ECON_SCORE_ALT_2
[[837.58389262  -7.62416107]
 [ -7.62416107 837.58389262]]


ECON_SCORE_ALT_3
[[837.58389262 447.24832215]
 [447.24832215 837.58389262]]


ECON_SCORE_ALT_4
[[837.58389262 559.7852349 ]
 [559.7852349  837.58389262]]




In [38]:
for x in ['ECON_SCORE_OG', 'ECON_SCORE_ALT_1', 'ECON_SCORE_ALT_2', 'ECON_SCORE_ALT_3', 'ECON_SCORE_ALT_4']:
    X = sm.add_constant(df[x])
    Y = df['RANK_GDP_EU']
    model = sm.OLS(Y, X)
    results_log_csm_nasa = model.fit(cov_type='HC1')
    print(results_log_csm_nasa.summary())

                            OLS Regression Results                            
Dep. Variable:            RANK_GDP_EU   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                   0.04116
Date:                Tue, 17 Oct 2023   Prob (F-statistic):              0.840
Time:                        18:34:57   Log-Likelihood:                -717.11
No. Observations:                 150   AIC:                             1438.
Df Residuals:                     148   BIC:                             1444.
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            51.1200      4.744     10.775

In [66]:
df_rank = df[['NUTS_ID', 'NAME_LATN', 'YEAR']]
df_rank['RANK_SCORE_OG'] = df.groupby(['YEAR'])['ECON_SCORE_OG'].rank()
df_rank['RANK_SCORE_ALT_1'] = df.groupby(['YEAR'])['ECON_SCORE_ALT_1'].rank()
df_rank['RANK_SCORE_ALT_2'] = df.groupby(['YEAR'])['ECON_SCORE_ALT_2'].rank()
df_rank['RANK_SCORE_ALT_3'] = df.groupby(['YEAR'])['ECON_SCORE_ALT_3'].rank()
df_rank['RANK_SCORE_ALT_4'] = df.groupby(['YEAR'])['ECON_SCORE_ALT_4'].rank()
df_rank['RANK_GDP'] = df.groupby(['YEAR'])['GDP_EU_MIL_EUR_2021'].rank()
df_rank['RANK_GDPPC'] = df.groupby(['YEAR'])['GDPPC_EU_EUR_2021'].rank()
df_rank

,NUTS_ID,NAME_LATN,YEAR,RANK_SCORE_OG,RANK_SCORE_ALT_1,RANK_SCORE_ALT_2,RANK_SCORE_ALT_3,RANK_SCORE_ALT_4,RANK_GDP,RANK_GDPPC
0,RS110,City of Belgrade,2017,13.0,25.0,11.0,25.0,25.0,25.0,25.0
1,RS110,City of Belgrade,2018,20.0,25.0,7.0,25.0,25.0,25.0,25.0
2,RS110,City of Belgrade,2019,17.0,25.0,16.0,25.0,25.0,25.0,25.0
3,RS110,City of Belgrade,2020,13.0,25.0,21.0,25.0,25.0,25.0,25.0
4,RS110,City of Belgrade,2021,12.0,25.0,12.0,25.0,25.0,25.0,25.0
...,...,...,...,...,...,...,...,...,...,...
145,RS229,Toplička oblast,2018,23.0,1.0,5.0,9.0,3.0,1.0,6.0
146,RS229,Toplička oblast,2019,20.0,2.0,20.0,6.0,3.0,1.0,6.0
147,RS229,Toplička oblast,2020,17.0,3.0,23.0,11.0,3.0,1.0,6.0
148,RS229,Toplička oblast,2021,18.0,5.0,14.0,10.0,4.0,1.0,6.0


In [68]:
for x in ['RANK_SCORE_OG', 'RANK_SCORE_ALT_1', 'RANK_SCORE_ALT_2', 'RANK_SCORE_ALT_3', 'RANK_SCORE_ALT_4']:
    print(x)    
    print(spearmanr(df_rank[x], df_rank['RANK_GDP']))
    print('\n')

RANK_SCORE_OG
SignificanceResult(statistic=0.016923076923076923, pvalue=0.8371460533245417)


RANK_SCORE_ALT_1
SignificanceResult(statistic=0.8146153846153847, pvalue=7.783589304325791e-37)


RANK_SCORE_ALT_2
SignificanceResult(statistic=-0.009102564102564104, pvalue=0.9119709258869143)


RANK_SCORE_ALT_3
SignificanceResult(statistic=0.533974358974359, pvalue=1.969875228636274e-12)


RANK_SCORE_ALT_4
SignificanceResult(statistic=0.6683333333333333, pvalue=9.298872512533859e-21)




In [69]:
for x in ['RANK_SCORE_OG', 'RANK_SCORE_ALT_1', 'RANK_SCORE_ALT_2', 'RANK_SCORE_ALT_3', 'RANK_SCORE_ALT_4']:
    print(x)    
    print(np.cov(df_rank[x], df_rank['RANK_GDP']))
    print('\n')

RANK_SCORE_OG
[[52.34899329  0.88590604]
 [ 0.88590604 52.34899329]]


RANK_SCORE_ALT_1
[[52.34899329 42.6442953 ]
 [42.6442953  52.34899329]]


RANK_SCORE_ALT_2
[[52.34899329 -0.47651007]
 [-0.47651007 52.34899329]]


RANK_SCORE_ALT_3
[[52.34899329 27.95302013]
 [27.95302013 52.34899329]]


RANK_SCORE_ALT_4
[[52.34899329 34.98657718]
 [34.98657718 52.34899329]]




In [45]:
for x in ['RANK_SCORE_OG', 'RANK_SCORE_ALT_1', 'RANK_SCORE_ALT_2', 'RANK_SCORE_ALT_3', 'RANK_SCORE_ALT_4']:
    X = sm.add_constant(df_rank[x])
    Y = df_rank['RANK_GDP']
    model = sm.OLS(Y, X)
    results = model.fit(cov_type='HC1')
    print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               RANK_GDP   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                   0.04116
Date:                Tue, 17 Oct 2023   Prob (F-statistic):              0.840
Time:                        18:45:07   Log-Likelihood:                -509.16
No. Observations:                 150   AIC:                             1022.
Df Residuals:                     148   BIC:                             1028.
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            12.7800      1.186     10.775

In [51]:
year = pd.Categorical(df_rank.YEAR)
X = df_rank.set_index(['NUTS_ID', 'YEAR'])
X['YEAR'] = year
Y = X['RANK_GDP']
model = lm.PanelOLS(Y, X[['RANK_SCORE_ALT_1', 'YEAR']], entity_effects=True)
full_pct = model.fit(cov_type='clustered', cluster_entity=True)
print(full_pct.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:               RANK_GDP   R-squared:                        0.0053
Estimator:                   PanelOLS   R-squared (Between):             -0.0515
No. Observations:                 150   R-squared (Within):               0.0053
Date:                Tue, Oct 17 2023   R-squared (Overall):             -0.0514
Time:                        18:48:06   Log-likelihood                   -167.45
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      0.1059
Entities:                          25   P-value                           0.9956
Avg Obs:                       6.0000   Distribution:                   F(6,119)
Min Obs:                       6.0000                                           
Max Obs:                       6.0000   F-statistic (robust):             0.4360
                            

## Population

In [72]:
df_rank['RANK_POP'] = df.groupby(['YEAR'])['POPULATION'].rank()
df_rank

,NUTS_ID,NAME_LATN,YEAR,RANK_SCORE_OG,RANK_SCORE_ALT_1,RANK_SCORE_ALT_2,RANK_SCORE_ALT_3,RANK_SCORE_ALT_4,RANK_GDP,RANK_GDPPC,RANK_POP
0,RS110,City of Belgrade,2017,13.0,25.0,11.0,25.0,25.0,25.0,25.0,25.0
1,RS110,City of Belgrade,2018,20.0,25.0,7.0,25.0,25.0,25.0,25.0,25.0
2,RS110,City of Belgrade,2019,17.0,25.0,16.0,25.0,25.0,25.0,25.0,25.0
3,RS110,City of Belgrade,2020,13.0,25.0,21.0,25.0,25.0,25.0,25.0,25.0
4,RS110,City of Belgrade,2021,12.0,25.0,12.0,25.0,25.0,25.0,25.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...
145,RS229,Toplička oblast,2018,23.0,1.0,5.0,9.0,3.0,1.0,6.0,2.0
146,RS229,Toplička oblast,2019,20.0,2.0,20.0,6.0,3.0,1.0,6.0,2.0
147,RS229,Toplička oblast,2020,17.0,3.0,23.0,11.0,3.0,1.0,6.0,2.0
148,RS229,Toplička oblast,2021,18.0,5.0,14.0,10.0,4.0,1.0,6.0,2.0


In [74]:
df_rank_pop = df_rank[df_rank['YEAR'] < 2021].reset_index(drop=True)

In [75]:
for x in ['RANK_SCORE_OG', 'RANK_SCORE_ALT_1', 'RANK_SCORE_ALT_2', 'RANK_SCORE_ALT_3', 'RANK_SCORE_ALT_4']:
    print(x)    
    print(np.cov(df_rank_pop[x], df_rank_pop['RANK_POP']))
    print('\n')

RANK_SCORE_OG
[[52.52525253 11.57575758]
 [11.57575758 52.52525253]]


RANK_SCORE_ALT_1
[[52.52525253 46.66666667]
 [46.66666667 52.52525253]]


RANK_SCORE_ALT_2
[[52.52525253 -3.45454545]
 [-3.45454545 52.52525253]]


RANK_SCORE_ALT_3
[[52.52525253 32.81818182]
 [32.81818182 52.52525253]]


RANK_SCORE_ALT_4
[[52.52525253 40.66666667]
 [40.66666667 52.52525253]]




In [76]:
for x in ['RANK_SCORE_OG', 'RANK_SCORE_ALT_1', 'RANK_SCORE_ALT_2', 'RANK_SCORE_ALT_3', 'RANK_SCORE_ALT_4']:
    print(x)    
    print(spearmanr(df_rank_pop[x], df_rank_pop['RANK_POP']))
    print('\n')

RANK_SCORE_OG
SignificanceResult(statistic=0.22038461538461537, pvalue=0.027573356223525424)


RANK_SCORE_ALT_1
SignificanceResult(statistic=0.8884615384615385, pvalue=6.434814010288862e-35)


RANK_SCORE_ALT_2
SignificanceResult(statistic=-0.06576923076923077, pvalue=0.5156095918528121)


RANK_SCORE_ALT_3
SignificanceResult(statistic=0.6248076923076923, pvalue=3.7210961504184716e-12)


RANK_SCORE_ALT_4
SignificanceResult(statistic=0.7742307692307693, pvalue=3.5039720197615976e-21)




In [77]:
for x in ['RANK_SCORE_OG', 'RANK_SCORE_ALT_1', 'RANK_SCORE_ALT_2', 'RANK_SCORE_ALT_3', 'RANK_SCORE_ALT_4']:
    X = sm.add_constant(df_rank_pop[x])
    Y = df_rank_pop['RANK_POP']
    model = sm.OLS(Y, X)
    results = model.fit(cov_type='HC1')
    print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               RANK_POP   R-squared:                       0.049
Model:                            OLS   Adj. R-squared:                  0.039
Method:                 Least Squares   F-statistic:                     5.172
Date:                Tue, 17 Oct 2023   Prob (F-statistic):             0.0251
Time:                        23:28:19   Log-Likelihood:                -336.97
No. Observations:                 100   AIC:                             677.9
Df Residuals:                      98   BIC:                             683.1
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            10.1350      1.447      7.003